## Prepare the dataset

In [1]:
from datasets import concatenate_datasets, load_dataset
raw_datasets = load_dataset("bookcorpus", split="train")
# wiki = load_dataset("wikipedia", "20220301.en", split="train")
# wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])  # only keep the 'text' column

# assert bookcorpus.features.type == wiki.features.type
# raw_datasets = concatenate_datasets([bookcorpus, wiki])

Found cached dataset bookcorpus (/root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f)


In [2]:
print(len(raw_datasets))

74004228


In [25]:
from datasets import Dataset

temp = {"text": [1,2,3,4,5]}
temp1 = Dataset.from_dict(temp)
print(type(temp1))

<class 'datasets.arrow_dataset.Dataset'>


In [5]:
print(type(raw_datasets))

<class 'datasets.arrow_dataset.Dataset'>


In [6]:
print(raw_datasets[0])

{'text': 'usually , he would be tearing around the living room , playing with his toys .'}


In [2]:
from transformers import AutoTokenizer
import multiprocessing
# from tqdm import tqdm

'''
## training a tokenizer from scratch
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(raw_datasets), batch_size)):
#         yield raw_datasets[i:i+batch_size]["text"]
        
# old_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = old_tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=30522)
# tokenizer.save_pretrained("tokenizer")
'''

tokenizer = AutoTokenizer.from_pretrained("tokenizer")
num_proc = multiprocessing.cpu_count()

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples["text"], return_special_tokens_mask=True, truncation=True, max_length=tokenizer.model_max_length
    )
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = raw_datasets.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features

Loading cached processed dataset at /root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f/cache-ddaf5e5442cf907f_*_of_00064.arrow


{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'special_tokens_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [3]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= tokenizer.model_max_length:
        total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=num_proc)
# shuffle dataset
tokenized_datasets = tokenized_datasets.shuffle(seed=34)

print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")

Loading cached processed dataset at /root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f/cache-63d4031cf6cf3571_*_of_00064.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f/cache-dad49d059028260c.arrow


the dataset contains in total 1239030784 tokens


In [4]:
from transformers import AutoConfig

model_config = AutoConfig.from_pretrained('bert-base-uncased')

In [5]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased', config=model_config, cache_dir="./BERT_cache")
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(15000, 768)

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./BERT",
    overwrite_output_dir = True,
    do_train = True,
    do_eval = True,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    logging_steps = 50,
    prediction_loss_only = True,
    learning_rate = 5e-5,
    weight_decay = 0,
    adam_epsilon = 1e-8,
    max_grad_norm = 1.0,
    num_train_epochs = 1,
    save_steps = -1
)

## Train a Tokenizer

In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=True, 
                                                mlm_probability=0.15,)

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

In [10]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,12.178000
2,9.012200
3,8.871000
4,10.278800
5,8.616600
6,8.454800
7,8.396900
8,8.270100
9,8.145000
10,8.027700
